<a href="https://colab.research.google.com/github/OMKARTT/Balancing-Dataset-Segmentation-/blob/main/Balancing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://conservancy.umn.edu/bitstream/handle/11299/214865/dataset.zip?sequence=12&isAllowed=y

--2023-02-25 11:12:57--  https://conservancy.umn.edu/bitstream/handle/11299/214865/dataset.zip?sequence=12
Resolving conservancy.umn.edu (conservancy.umn.edu)... 128.101.122.105
Connecting to conservancy.umn.edu (conservancy.umn.edu)|128.101.122.105|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 553029970 (527M) [application/zip]
Saving to: ‘dataset.zip?sequence=12’

dataset.zip?sequenc 100%[===================>] 527.41M  4.63MB/s    in 3m 38s  

2023-02-25 11:16:36 (2.42 MB/s) - ‘dataset.zip?sequence=12’ saved [553029970/553029970]



In [ ]:
!cp dataset.zip?sequence=12 dataset.zip
!rm dataset.zip?sequence=12
!unzip dataset.zip

In [3]:
!git clone https://github.com/philferriere/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 994, done.
remote: Total 994 (delta 0), reused 0 (delta 0), pack-reused 994
Receiving objects: 100% (994/994), 12.26 MiB | 3.26 MiB/s, done.
Resolving deltas: 100% (589/589), done.


In [4]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import random
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [5]:
dataDir='/content/dataset/material_version'
dataType='train'
annFile='/content/dataset/material_version/instances_train_trashcan.json'.format(dataDir,dataType)

# Initialize the COCO api for instance annotations
coco=COCO(annFile)

# Load the categories in a variable
catIDs = coco.getCatIds()
cats = coco.loadCats(catIDs)
print(len(cats))
print(cats)
print(catIDs)

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
16
[{'supercategory': 'rov', 'id': 1, 'name': 'rov'}, {'supercategory': 'plant', 'id': 2, 'name': 'plant'}, {'supercategory': 'animal_fish', 'id': 3, 'name': 'animal_fish'}, {'supercategory': 'animal_starfish', 'id': 4, 'name': 'animal_starfish'}, {'supercategory': 'animal_shells', 'id': 5, 'name': 'animal_shells'}, {'supercategory': 'animal_crab', 'id': 6, 'name': 'animal_crab'}, {'supercategory': 'animal_eel', 'id': 7, 'name': 'animal_eel'}, {'supercategory': 'animal_etc', 'id': 8, 'name': 'animal_etc'}, {'supercategory': 'trash_etc', 'id': 9, 'name': 'trash_etc'}, {'supercategory': 'trash_fabric', 'id': 10, 'name': 'trash_fabric'}, {'supercategory': 'trash_fishing_gear', 'id': 11, 'name': 'trash_fishing_gear'}, {'supercategory': 'trash_metal', 'id': 12, 'name': 'trash_metal'}, {'supercategory': 'trash_paper', 'id': 13, 'name': 'trash_paper'}, {'supercategory': 'trash_plastic', 'id': 14, 'name': 'trash

In [6]:
def getClassName(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"
print('The class name is', getClassName(2, cats))

The class name is plant


In [7]:
# Define the classes (out of the 81) which you want to see. Others will not be shown.
filterClasses = ['none']

# Fetch class IDs only corresponding to the filterClasses
catIds = coco.getCatIds(catNms=filterClasses) 
# Get all images containing the above Category IDs
imgIds = coco.getImgIds(catIds=catIds)
print("Number of images containing all the  classes:", len(imgIds))


Number of images containing all the  classes: 6008


In [8]:
os.mkdir("/content/dataset/material_version/train_mask")

In [9]:
def pixel_value(index):
  #print("Categoory",index)
  if index==1:    #rov
    return 1
  elif index==2:   # plant
    return 2
  elif index>=3 and index<=8:   #animal
    return 3
  elif index>=9 and index<=16:  #trash
    return 4

In [10]:
fix=20

In [11]:
clcount=[0]*17

In [12]:
clscount=[0,150,150,20,20,20,20,20,20,20,20,20,20,20,20,20,20]
clscount[1]

150

In [13]:
def class_count(count):
  global clcount
  if clcount[count]<=clscount[count]:
    clcount[count]=clcount[count]+1
    return 0
  else:
    return 1

In [14]:
count1=0
count2=0
count3=0
count4=0

In [16]:
def classcount(mul):
  global count1
  global count2
  global count3
  global count4
  if mul==1:
    count1=count1+1
    return 0
  elif mul==2 :
    count2=count2+1
    return 0
  elif mul==3:
    count3=count3+1
    return 0
  elif mul==4:
    count4=count4+1
    return 0
  else:
    return 1

In [17]:
def conversion(i):
  filterClasses = ['rov','plant','animal_fish','animal_starfish','animal_shells',
                  'animal_crab','animal_eel','animal_etc','trash_etc','trash_fabric',
                 'trash_fishing_gear','trash_metal','trash_paper','trash_plastic',
                 'trash_rubber','trash_wood']
  img = coco.loadImgs(imgIds[i])[0]
  print(img)
  I = io.imread('{}/{}/{}'.format(dataDir,dataType,img['file_name']))/255.0
  annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
  #print(img['file_name'])
  #print(img['id'])
  anns = coco.loadAnns(annIds)
  mask = np.zeros((img['height'],img['width']))
  for a in range(len(anns)):
    global flag
    flag=1
    className = getClassName(anns[a]['category_id'], cats)
    #print(className)
    classId= filterClasses.index(className)+1
    mul=pixel_value(classId)
    #m=maskimg(i,mul,mask)
    categorycount=classcount(mul)
    flag =class_count(classId)
    print(flag)
    if flag==0:
      mask = np.maximum(coco.annToMask(anns[a])*mul, mask)
      cv2.imwrite("/content/dataset/material_version/train_mask/"+img['file_name'],mask);


In [ ]:
import cv2
import os
import numpy as np
from tqdm.notebook import tqdm
for i in tqdm(range(6008)):
  conversion(i)